### Building a Project Management Assistant with OpenAI API Function Calling

In this demo, we will explore the power of OpenAI's API function calling capabilities by building a project management assistant. This assistant will interact with a simulated project management database, to perform various tasks such as updating task statuses, generating project reports, and managing resource allocation.


#### What is Function Calling?

Function calling allows AI models to interact with external tools and APIs by generating structured JSON outputs that represent function calls. This enables the model to perform actions beyond text generation, such as retrieving information, performing calculations, or controlling external systems.

- Basic Structure

```json
{
  "tools": [
    {
      "type": "function",
      "function": {
        "name": "simple_function",
        "description": "A simple function that returns a string",
        "parameters": {
          "type": "object",
          "properties": {
            "input_string": {
              "type": "string",
              "description": "A string to pass to the function"
            }
          },
          "required": ["input_string"]
        }
      }
    }
  ]
}
```

#### Key Components Explained

1. **Tools Array**: Contains all available tools/functions the model can call
   - Each tool is defined as an object in this array

2. **Type**: Specifies the kind of tool (currently "function" is the standard type)

3. **Function Object**: Contains the function's metadata
   - **name**: The identifier used to call the function
   - **description**: Explains what the function does (helps the model decide when to use it)
   - **parameters**: Defines the expected input format using JSON Schema

4. **Parameters Object**: Uses JSON Schema to define expected inputs
   - **type**: Usually "object" for structured parameters
   - **properties**: Defines each individual parameter
   - **required**: Lists which parameters must be provided

5. **Individual Parameters**: Each defined with:
   - **type**: Data type (string, number, boolean, array, object)
   - **description**: Explains the parameter's purpose

#### How Function Calling Works

1. The model receives a prompt from the user
2. Based on the prompt, the model determines if a function call is needed
3. If needed, the model generates a structured function call with appropriate parameters
4. The application intercepts this function call and executes the actual function
5. The function's result is returned to the model
6. The model incorporates the result into its response to the user

#### Example Usage Flow

User: "What's the weather in Paris?"

Model (internal reasoning): *This requires current weather data I don't have. I should use a weather function.*

Model (generates function call):
```json
{
  "name": "get_weather",
  "arguments": "{\"location\": \"Paris\", \"unit\": \"celsius\"}"
}
```

System executes `get_weather("Paris", "celsius")` and returns: `{"temperature": 22, "condition": "Sunny"}`

Model (final response): "The weather in Paris is currently sunny with a temperature of 22°C."


### Import necessary libraries

In [1]:
import os
import dotenv
import json
from openai import OpenAI
import pandas as pd

dotenv.load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("OPENAI_BASE_URL")

In [7]:
client = OpenAI(
	api_key=api_key,
	base_url=base_url
)

### Define Functions

First, define the Python functions that will read and write from the project_management.csv file using Pandas dataframes. This code uses Pandas dataframes to read and write from the .csv file. 

We define 3 tasks our project management assistant can perform. 

Each function returns a JSON string as output

In [2]:
# Load the project management data
df = pd.read_csv('project_management.csv')
df

,Task ID,Task Name,Project ID,Assigned To,Status,Priority,Due Date,Date Created,Last Updated,Time Estimate,Time Spent,Description,Project Phase,Dependencies
0,1,Design Database Schema,101,Jane Doe,In Progress,High,2023-08-01,2023-07-01,2023-07-10,10,4.0,Create initial database schema for customer data,Design,NaN
1,2,Implement Login Page,101,John Smith,completed,Medium,2023-08-15,2023-07-01,2023-12-15,5,NaN,"""Develop the login page UI and backend""",Implementation,1.0
2,3,Prepare Project Report,102,Alice Johnson,Completed,Low,2023-07-15,2023-06-01,2023-07-05,2,2.0,Compile the weekly project status report,Reporting,NaN


In [4]:
def task_retrieval_and_status_updates(task_id, status, last_updated):
    """Retrieve and update task status"""
    df.loc[df['Task ID'] == task_id, 'Status'] = status
    df.loc[df['Task ID'] == task_id, 'Last Updated'] = last_updated
    df.to_csv('project_management.csv', index=False)  # save changes to file
    task = df.loc[df['Task ID'] == task_id]
    return json.dumps(task.to_dict())

def project_reporting_and_analytics(project_id):
    """Generate reports on project progress and team performance"""
    project = df.loc[df['Project ID'] == project_id]
    return json.dumps(project.to_dict())

def resource_allocation_and_scheduling(task_id, assigned_to, time_estimate, due_date, status):
    """Allocate tasks based on current workloads and schedules"""
    df.loc[df['Task ID'] == task_id, 'Assigned To'] = assigned_to
    df.loc[df['Task ID'] == task_id, 'Time Estimate'] = time_estimate
    df.loc[df['Task ID'] == task_id, 'Due Date'] = due_date
    df.loc[df['Task ID'] == task_id, 'Status'] = status
    df.to_csv('project_management.csv', index=False)  # save changes to file
    task = df.loc[df['Task ID'] == task_id]
    return json.dumps(task.to_dict())

Next, we'll build the project management assistant conversation. 

We'll define the messages to send to the model, including a tools dictionary that defines a list of tools, which are the functions that are available to the model to identify and parse parameters for. 

In [20]:
def run_conversation(user_prompt):
    messages = [
        {
            "role": "system", 
            "content": "You are a project management assistant with knowledge of project statuses, task assignments, and scheduling. You can provide updates on projects, assign tasks to team members, and schedule meetings. You understand project management terminology and are capable of parsing detailed project data. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
        },
        {
            "role": "user", 
            "content": user_prompt
        } # this prompt should call task_retrieval_and_status_updates
    ]
    
    tools= [
        {
            "type": "function",
            "function": {
                "name": "task_retrieval_and_status_updates",
                "description": "Retrieve and update task status",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "task_id": {
                            "type": "integer",
                            "description": "The unique identifier for the task"
                        },
                        "status": {
                            "type": "string",
                            "description": "The new status of the task"
                        },
                        "last_updated": {
                            "type": "string",
                            "description": "The date of the last status update or change to the task"
                        }
                    },
                    "required": ["task_id", "status", "last_updated"]
                }
            }
        },
        {
            "type": "function",
            # Fill in the function definition for the project_reporting_and_analytics function
            "function": {
                "name": "project_reporting_and_analytics",
                "description": "Generate reports on project progress and team performance",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "project_id": {
                            "type": "integer",
                            "decription": "The unique identifier for the project"
                        }
                    },
                    "required": ["project_id"]
                }
             
            }
        },
        {
            "type": "function",
            # Fill in the function definition for the resource_allocation_and_scheduling function
            "function": {
                "name": "resource_allocation_and_scheduling",
                "description": "Allocate tasks based on current workloads and schedules",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "task_id": {
                            "type": "integer",
                            "description": "The unique identifier for the task"
                        },
                        "assigned_to": {
                            "type": "string",
                            "description": "The user be assigned to certain task"
                        },
                        "time_estimate": {
                            "type": "integer",
                            "description": "Finish the task will consume how many days"
                        },
                        "due_date": {
                            "type": "string",
                            "format": "date-time",
                            "description": "The last day to complete the task"
                        },
                        "status": {
                            "type": "string",
                            "description": "The task status, such as 'Completed' or 'In Progress'"
                        }
                    },
                    "required": ["task_id", "assigned_to", "time_estimate", "due_date", "status"]
                }
            }
        }
    ]
    # Use openai.ChatCompletion.create for openai < 1.0
    # openai.chat.completions.create for openai > 1.0
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # let the model decide which tool (function) to use
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls # get the tool calls from the first response
    print(tool_calls)
    # end of first response, now we parse the response and call the functions the model identified from our tool list
    # check if the model wanted to call a function
    if tool_calls:
        # list the available functions and their corresponding python functions
        available_functions = {
            "task_retrieval_and_status_updates": task_retrieval_and_status_updates,
            "project_reporting_and_analytics": project_reporting_and_analytics,
            "resource_allocation_and_scheduling": resource_allocation_and_scheduling,
        }  
        messages.append(response_message)  # extend the conversation with the first response
        # send the info for each function call and function response to the model
        for tool_call in tool_calls: # iterate through the tool calls in the response
            function_name = tool_call.function.name # get the name of the function to call
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments) 
            if function_name == 'task_retrieval_and_status_updates': # call the Python function
                function_response = function_to_call(
                    task_id=function_args.get("task_id"),
                    status=function_args.get("status"),
                    last_updated=function_args.get("last_updated")
                )
            elif function_name == 'project_reporting_and_analytics':
                # Fill in the function call for the project_reporting_and_analytics function with parameters from the function_args dictionary
                function_response = function_to_call(
                    project_id=function_args.get("project_id")
                )

            elif function_name == 'resource_allocation_and_scheduling':
                # Fill in the function call for the resource_allocation_and_scheduling function with parameters from the function_args dictionary
                function_response = function_to_call(
                    task_id=function_args.get("task_id"),
                    assigned_to=function_args.get("assigned_to"),
                    time_estimate=function_args.get("time_estimate"),
                    due_date=function_args.get("due_date"),
                    status=function_args.get("status")
                )

            message_to_append = {
                    "tool_call_id": tool_call.id, #
                    "role": "tool",
                    "name": function_name,
                    "content": function_response, # send the function response to the model, it's the JSON string of the function response
                }
            messages.append(message_to_append)  # extend conversation with function response

        # See https://gist.github.com/gaborcselle/2dc076eae23bd219ff707b954c890cd7
        messages[1]['content'] = "" # clear the first message (parsing bug)

        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
    
print(run_conversation("Change the status of task 2 to completed.")) # will print the second response from the model

[ChatCompletionMessageToolCall(id='call_PSot8JJF3DdeVVnJoF5EZR3X', function=Function(arguments='{"task_id":2,"status":"Completed","last_updated":"2022-01-15"}', name='task_retrieval_and_status_updates'), type='function')]
ChatCompletion(id='chatcmpl-BLEFbXOIwd3O9QxbjcfZPOEA9m5ZM', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The task "Implement Login Page" with Task ID 2 has been updated to status "Completed" as of January 15, 2022. If there are any further updates or tasks to be assigned, feel free to let me know!', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1744399367, model='gpt-3.5-turbo-1106', object='chat.completion', service_tier='default', system_fingerprint='fp_982035f36f', usage=CompletionUsage(completion_tokens=50, prompt_tokens=293, total_tokens=343, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_

### Best Practices

1. Provide clear function descriptions to help the model understand when to use them
2. Use specific parameter descriptions to guide proper parameter usage
3. Mark parameters as required only when absolutely necessary
4. Design functions to be atomic and focused on specific tasks
5. Handle errors gracefully when the model provides invalid parameters

### Implementation Considerations

- Function calling requires a runtime that can parse the model's JSON output
- The application must implement the actual functions being called
- Consider rate limits and API costs for external service calls
- Add validation to ensure security when executing function calls
